# Simulation 02 -- Symmetric Distortion Stress Test

## Question
**How much symmetric noise can the pseudo-label approach tolerate before it breaks?**

## Background
In Simulation 01 we confirmed the pipeline can recover FMV under the default 10% origination noise.
Here we stress-test the approach by **systematically increasing** the origination noise from 5% to 50%
and measuring how each pipeline stage degrades.

## Approach
1. Regenerate `current_monthly_rent` at each noise level (keeping all features and true FMV fixed)
2. Run the full 4-stage pipeline at each level
3. Record metrics: MAPE, MAE, R², classification accuracy, core %, within-cluster CV
4. Find the **breaking point** where MAPE crosses 20%
5. Check whether defense layers (Isolation Forest, land value correlation) detect the degradation

## Noise Levels
| Level | `noise_std` | Interpretation |
|-------|-------------|----------------|
| 1     | 0.05        | Very low noise -- nearly perfect origination |
| 2     | 0.10        | Default (same as sim_00) |
| 3     | 0.20        | Moderate -- 20% origination spread |
| 4     | 0.30        | High -- 30% spread |
| 5     | 0.50        | Extreme -- 50% spread |

In [ ]:
# ============================================================
# Imports & Configuration
# ============================================================
import numpy as np
import pandas as pd
from scipy import stats
from scipy.stats import median_abs_deviation
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import GroupKFold
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor, IsolationForest
from sklearn.metrics import (
    mean_absolute_error, mean_squared_error, r2_score,
    confusion_matrix, accuracy_score, cohen_kappa_score,
    silhouette_score,
)

from xgboost import XGBRegressor

# Quantile forest -- graceful fallback
try:
    from quantile_forest import RandomForestQuantileRegressor
    HAS_QF = True
    print('\u2713 quantile-forest available -- using RandomForestQuantileRegressor')
except ImportError:
    HAS_QF = False
    print('\u2717 quantile-forest not installed -- falling back to sklearn RF with manual quantile estimation')

# HDBSCAN -- try sklearn >= 1.3, then hdbscan library
try:
    from sklearn.cluster import HDBSCAN as SklearnHDBSCAN
    HAS_SKLEARN_HDBSCAN = True
    print('\u2713 Using sklearn.cluster.HDBSCAN')
except ImportError:
    HAS_SKLEARN_HDBSCAN = False
    import hdbscan
    print('\u2713 Using hdbscan library')

sns.set_style('whitegrid')
plt.rcParams['figure.dpi'] = 110

print('\nAll imports loaded successfully.')

---
## Load Data & Define Feature Sets

We load the same synthetic dataset as Simulation 01 and define identical feature lists.
The key difference: we will **regenerate** `current_monthly_rent` at each noise level rather
than using the default distorted rents.

In [ ]:
# ============================================================
# Load synthetic data
# ============================================================
df_raw = pd.read_parquet('../data/synthetic/synthetic_5k_sites.parquet')
print(f'Loaded {len(df_raw):,} sites  |  {len(df_raw.columns)} columns')
print(f'True FMV range: ${df_raw["true_fmv"].min():,.0f} -- ${df_raw["true_fmv"].max():,.0f}')
df_raw.head(3)

In [ ]:
# ============================================================
# Define feature lists (identical to sim_01)
# ============================================================

# X_MARKET: features available for ALL sites (clustering + supervised).
# No rent, no Z_NETWORK in clustering.
X_MARKET = [
    # Demographics & Market (Domain 5)
    'census_population_density', 'census_population_3km',
    'census_median_income', 'census_median_home_value',
    'census_median_rent', 'census_vacancy_rate',
    'urban_rural_class', 'underlying_land_value_psf',
    'hud_safmr', 'property_tax_rate',
    # Topography / Geography
    'topography_class', 'terrain_elevation_variance_m',
    'poi_density_1km', 'building_density_500m',
    'distance_to_highway_km', 'ground_elevation_ft',
    'unemployment_rate_local',
    # Competition / Supply (Domain 7)
    'fcc_tower_count_5km', 'fcc_tower_density_sqkm',
    'competitor_tower_distance_km', 'tower_per_capita',
    'scarcity_index', 'landlord_concentration',
    'composite_hazard_score',
    # Property / Structure (Domain 1 & 3)
    'tower_height_ft', 'lease_area_sqft', 'parcel_acreage',
    'construction_age', 'site_access_ordinal',
    # Zoning / Regulatory (Domain 2)
    'permit_approval_difficulty', 'wrluri_index',
    'colocation_required', 'height_restriction_ft',
    'has_building', 'is_ground_tower', 'has_parcel', 'has_zoning_data',
]

# Z_NETWORK: carrier/network features (for supervised model only)
Z_NETWORK = [
    'tenant_count', 'primary_technology', 'coverage_critical',
    'capacity_utilization_pct', 'high_traffic', 'backhaul_type',
    'has_fiber_connection', 'network_substitutability_score',
    'search_ring_alternative_count', 'inter_site_distance_km',
    'antenna_count', 'structure_capacity_remaining',
    'site_priority_tier',
]

ALL_FEATURES = X_MARKET + Z_NETWORK

NOISE_LEVELS = [0.05, 0.10, 0.20, 0.30, 0.50]

print(f'X_MARKET features:   {len(X_MARKET)}')
print(f'Z_NETWORK features:  {len(Z_NETWORK)}')
print(f'ALL_FEATURES:        {len(ALL_FEATURES)}')
print(f'Noise levels to test: {NOISE_LEVELS}')

In [ ]:
# ============================================================
# Encode categorical features (once, shared across all runs)
# ============================================================
le_structure = LabelEncoder()
df_raw['structure_type_enc'] = le_structure.fit_transform(df_raw['structure_type'])

le_zoning = LabelEncoder()
df_raw['zoning_district_type_enc'] = le_zoning.fit_transform(df_raw['zoning_district_type'])

prop_dummies = pd.get_dummies(df_raw['property_type'], prefix='pt')
df_raw = pd.concat([df_raw, prop_dummies], axis=1)

ENCODED_CATS = ['structure_type_enc', 'zoning_district_type_enc'] + list(prop_dummies.columns)
X_MARKET_ENC = X_MARKET + ENCODED_CATS
ALL_FEATURES_ENC = ALL_FEATURES + ENCODED_CATS

print(f'Encoded feature set sizes:')
print(f'  X_MARKET_ENC:     {len(X_MARKET_ENC)}')
print(f'  ALL_FEATURES_ENC: {len(ALL_FEATURES_ENC)}')

---
## Define the Full Pipeline Function

This function encapsulates the entire 4-stage pipeline from Simulation 01 so we can call it
repeatedly at different noise levels. At each invocation it:

1. **Regenerates** `current_monthly_rent` from `true_fmv` with the given `noise_std`:
   ```
   origination_factor = np.random.lognormal(0, noise_std, N)
   base_rent_at_inception = true_fmv_at_inception * origination_factor
   current_rent = base_rent_at_inception * cumulative_escalation_factor
   ```
2. **Clusters** sites with HDBSCAN on X_MARKET (features remain fixed)
3. **Pseudo-labels** with MAD threshold = 1.0
4. **Trains** XGBoost + QRF + Ridge meta-learner on CORE sites (GroupKFold on state)
5. **Predicts** FMV for all sites and classifies
6. **Returns** a metrics dictionary for the sweep table

In [ ]:
def run_pipeline(df, noise_std, seed=42, verbose=True):
    """
    Run the full FMV pipeline with regenerated rents at the given noise level.

    Parameters
    ----------
    df : pd.DataFrame
        Base dataset with all features, true_fmv, and lease parameters.
    noise_std : float
        Standard deviation of the log-normal origination noise.
    seed : int
        Random seed for reproducibility.
    verbose : bool
        Whether to print progress.

    Returns
    -------
    dict with keys: noise_std, mape, mae, rmse, r2, classification_accuracy,
                    core_pct, within_cluster_cv, n_clusters,
                    predicted_fmv, model_class, qrf_q25_all, qrf_q75_all, df
    """
    rng = np.random.RandomState(seed)
    df = df.copy()
    N = len(df)

    # ==========================================================
    # Step 0: Regenerate current_monthly_rent at new noise level
    # ==========================================================
    origination_factor = rng.lognormal(0, noise_std, N)

    historical_growth_rate = 0.02
    true_fmv_at_inception = (
        df['true_fmv'].values
        / (1 + historical_growth_rate) ** df['lease_age_years'].values
    )

    base_rent_at_inception = true_fmv_at_inception * origination_factor
    current_rent = base_rent_at_inception * df['cumulative_escalation_factor'].values
    current_rent = np.clip(current_rent, 100, 15000)
    df['current_monthly_rent'] = current_rent

    # Re-derive true class
    true_dev = (current_rent - df['true_fmv'].values) / df['true_fmv'].values
    df['true_class'] = np.where(
        true_dev > 0.15, 'OVERPAYING',
        np.where(true_dev < -0.15, 'UNDERPAYING', 'AT_MARKET')
    )

    if verbose:
        print(f'  noise_std={noise_std:.2f}: rent median=${np.median(current_rent):,.0f}, '
              f'std=${current_rent.std():,.0f}')

    # ==========================================================
    # Stage 1: HDBSCAN Clustering
    # ==========================================================
    scaler = StandardScaler()
    X_clust = scaler.fit_transform(df[X_MARKET_ENC].fillna(0))

    if HAS_SKLEARN_HDBSCAN:
        clusterer = SklearnHDBSCAN(
            min_cluster_size=20, min_samples=5,
            metric='euclidean', n_jobs=-1,
        )
    else:
        clusterer = hdbscan.HDBSCAN(
            min_cluster_size=20, min_samples=5,
            metric='euclidean', core_dist_n_jobs=-1,
        )

    df['cluster'] = clusterer.fit_predict(X_clust)
    n_clusters = df['cluster'].nunique() - (1 if -1 in df['cluster'].values else 0)

    # Within-cluster CV
    non_noise = df[df['cluster'] != -1]
    cluster_cvs = []
    for _, grp in non_noise.groupby('cluster'):
        if len(grp) >= 5:
            cv = grp['current_monthly_rent'].std() / grp['current_monthly_rent'].mean()
            cluster_cvs.append(cv)
    within_cluster_cv = np.mean(cluster_cvs) if cluster_cvs else np.nan

    # ==========================================================
    # Stage 2: Pseudo-labeling (MAD threshold = 1.0)
    # ==========================================================
    labels = pd.Series('MARGINAL', index=df.index)
    for cid, grp in df.groupby('cluster'):
        if cid == -1:
            continue
        rents = grp['current_monthly_rent']
        med = rents.median()
        mad = median_abs_deviation(rents, scale='normal')
        if mad < 1e-6:
            mad = rents.std() if rents.std() > 0 else 1.0
        z = np.abs(rents - med) / mad
        labels.loc[grp.index[z < 1.0]] = 'CORE'
        labels.loc[grp.index[(z >= 1.0) & (z < 2.0)]] = 'MARGINAL'
        labels.loc[grp.index[z >= 2.0]] = 'OUTLIER'

    df['pseudo_label'] = labels
    core_mask = df['pseudo_label'] == 'CORE'
    core_pct = core_mask.sum() / N

    # Safety net: if very few CORE sites, broaden to non-outlier
    if core_mask.sum() < 100:
        if verbose:
            print(f'    WARNING: Only {core_mask.sum()} CORE sites. '
                  f'Broadening to non-OUTLIER.')
        core_mask = df['pseudo_label'] != 'OUTLIER'
        core_pct = core_mask.sum() / N

    # ==========================================================
    # Stage 3: XGBoost + QRF with Ridge meta-learner
    # ==========================================================
    core_df = df[core_mask]
    X_tr = core_df[ALL_FEATURES_ENC].fillna(0).values
    y_tr = core_df['current_monthly_rent'].values
    groups = core_df['state'].values

    N_SPLITS = 5
    gkf = GroupKFold(n_splits=N_SPLITS)

    oof_xgb = np.full(len(core_df), np.nan)
    oof_qrf = np.full(len(core_df), np.nan)
    xgb_models = []
    qrf_models = []

    for fold_i, (tr_idx, val_idx) in enumerate(
        gkf.split(X_tr, y_tr, groups)
    ):
        Xf_tr, Xf_val = X_tr[tr_idx], X_tr[val_idx]
        yf_tr, yf_val = y_tr[tr_idx], y_tr[val_idx]

        # XGBoost
        xgb_m = XGBRegressor(
            n_estimators=500, learning_rate=0.05, max_depth=6,
            subsample=0.8, colsample_bytree=0.8, min_child_weight=5,
            reg_alpha=0.1, reg_lambda=1.0,
            random_state=seed + fold_i, n_jobs=-1, verbosity=0,
        )
        xgb_m.fit(Xf_tr, yf_tr, eval_set=[(Xf_val, yf_val)], verbose=False)
        oof_xgb[val_idx] = xgb_m.predict(Xf_val)
        xgb_models.append(xgb_m)

        # QRF
        if HAS_QF:
            qrf_m = RandomForestQuantileRegressor(
                n_estimators=500, min_samples_leaf=5,
                random_state=seed + fold_i, n_jobs=-1,
            )
            qrf_m.fit(Xf_tr, yf_tr)
            oof_qrf[val_idx] = qrf_m.predict(Xf_val, quantiles=[0.50])[:, 0]
        else:
            qrf_m = RandomForestRegressor(
                n_estimators=500, min_samples_leaf=5,
                random_state=seed + fold_i, n_jobs=-1,
            )
            qrf_m.fit(Xf_tr, yf_tr)
            tree_preds = np.array(
                [t.predict(Xf_val) for t in qrf_m.estimators_]
            )
            oof_qrf[val_idx] = np.percentile(tree_preds, 50, axis=0)

        qrf_models.append(qrf_m)

    # Ridge meta-learner
    valid_mask = ~np.isnan(oof_xgb) & ~np.isnan(oof_qrf)
    X_meta_tr = np.column_stack([oof_xgb[valid_mask], oof_qrf[valid_mask]])
    y_meta_tr = y_tr[valid_mask]
    meta = Ridge(alpha=1.0)
    meta.fit(X_meta_tr, y_meta_tr)

    # ==========================================================
    # Stage 4: Predict FMV for ALL sites
    # ==========================================================
    X_all = df[ALL_FEATURES_ENC].fillna(0).values

    xgb_preds = np.mean([m.predict(X_all) for m in xgb_models], axis=0)

    if HAS_QF:
        qrf_preds_list = []
        for m in qrf_models:
            qrf_preds_list.append(
                m.predict(X_all, quantiles=[0.10, 0.25, 0.50, 0.75, 0.90])
            )
        qrf_preds_all = np.mean(qrf_preds_list, axis=0)
        qrf_q25 = qrf_preds_all[:, 1]
        qrf_q50 = qrf_preds_all[:, 2]
        qrf_q75 = qrf_preds_all[:, 3]
    else:
        all_tree_preds = []
        for m in qrf_models:
            for tree in m.estimators_:
                all_tree_preds.append(tree.predict(X_all))
        all_tree_preds = np.array(all_tree_preds)
        qrf_q25 = np.percentile(all_tree_preds, 25, axis=0)
        qrf_q50 = np.percentile(all_tree_preds, 50, axis=0)
        qrf_q75 = np.percentile(all_tree_preds, 75, axis=0)

    # Meta-learner final prediction
    X_meta_all = np.column_stack([xgb_preds, qrf_q50])
    predicted_fmv = np.clip(meta.predict(X_meta_all), 100, 15000)

    # Classification
    model_class = np.where(
        current_rent > qrf_q75, 'OVERPAYING',
        np.where(current_rent < qrf_q25, 'UNDERPAYING', 'AT_MARKET')
    )

    # ==========================================================
    # Metrics vs ground truth
    # ==========================================================
    y_true = df['true_fmv'].values
    mae = mean_absolute_error(y_true, predicted_fmv)
    mape = np.mean(np.abs((y_true - predicted_fmv) / y_true)) * 100
    rmse = np.sqrt(mean_squared_error(y_true, predicted_fmv))
    r2 = r2_score(y_true, predicted_fmv)
    acc = accuracy_score(df['true_class'].values, model_class)

    if verbose:
        print(f'    -> MAPE={mape:.1f}%  R\u00b2={r2:.3f}  '
              f'Acc={acc:.1%}  Core={core_pct:.1%}')

    return {
        'noise_std': noise_std,
        'mape': mape,
        'mae': mae,
        'rmse': rmse,
        'r2': r2,
        'classification_accuracy': acc,
        'core_pct': core_pct,
        'within_cluster_cv': within_cluster_cv,
        'n_clusters': n_clusters,
        # Pass through for defense layer analysis
        'predicted_fmv': predicted_fmv,
        'model_class': model_class,
        'qrf_q25_all': qrf_q25,
        'qrf_q75_all': qrf_q75,
        'df': df,
    }

---
## Run Pipeline at All Noise Levels

This is the main experiment loop. For each noise level, we regenerate the distorted rents
(keeping all features and true FMV fixed), run the full pipeline, and collect metrics.
This lets us isolate the effect of origination noise on pipeline performance.

In [ ]:
# ============================================================
# Run pipeline at 5 noise levels
# ============================================================
results = []

for noise_std in NOISE_LEVELS:
    print(f'\n{"=" * 60}')
    print(f'Running pipeline with noise_std = {noise_std}')
    print(f'{"=" * 60}')
    res = run_pipeline(df_raw, noise_std, seed=42, verbose=True)
    results.append(res)

print(f'\n{"=" * 60}')
print(f'All {len(NOISE_LEVELS)} noise levels completed.')
print(f'{"=" * 60}')

---
## Summary Table

This table shows all pipeline metrics at each noise level, making it easy to spot the
degradation trajectory.

In [ ]:
# ============================================================
# Build summary table
# ============================================================
summary_cols = [
    'noise_std', 'mape', 'mae', 'rmse', 'r2',
    'classification_accuracy', 'core_pct', 'within_cluster_cv', 'n_clusters',
]
summary_df = pd.DataFrame([{k: r[k] for k in summary_cols} for r in results])

# Format for display
display_df = summary_df.copy()
display_df['mape'] = display_df['mape'].map('{:.1f}%'.format)
display_df['mae'] = display_df['mae'].map('${:,.0f}'.format)
display_df['rmse'] = display_df['rmse'].map('${:,.0f}'.format)
display_df['r2'] = display_df['r2'].map('{:.3f}'.format)
display_df['classification_accuracy'] = display_df['classification_accuracy'].map('{:.1%}'.format)
display_df['core_pct'] = display_df['core_pct'].map('{:.1%}'.format)
display_df['within_cluster_cv'] = display_df['within_cluster_cv'].map('{:.3f}'.format)

print('SYMMETRIC DISTORTION STRESS TEST -- SUMMARY')
print('=' * 100)
print(display_df.to_string(index=False))

---
## Degradation Curves

These plots show how each metric degrades as origination noise increases. The key question
is: **at what noise level does the pipeline "break"** (MAPE > 20%)?

We expect:
- MAPE increases monotonically with noise
- Classification accuracy decreases
- R² decreases
- Within-cluster CV increases (clusters become less homogeneous in rent)

In [ ]:
# ============================================================
# Plot degradation curves (2x2 grid)
# ============================================================
noise_vals = summary_df['noise_std'].values

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# --- MAPE vs noise ---
ax = axes[0, 0]
ax.plot(noise_vals, summary_df['mape'].values, 'o-', color='#e74c3c', lw=2, ms=8)
ax.axhline(15, color='green', ls='--', lw=1.5, alpha=0.7, label='GO threshold (15%)')
ax.axhline(20, color='red', ls='--', lw=1.5, alpha=0.7, label='Breaking point (20%)')
ax.set_xlabel('Origination Noise (std)', fontsize=11)
ax.set_ylabel('MAPE (%)', fontsize=11)
ax.set_title('MAPE vs Noise Level', fontsize=12)
ax.legend(fontsize=9)
ax.grid(True, alpha=0.3)

# --- Classification Accuracy vs noise ---
ax = axes[0, 1]
ax.plot(noise_vals, summary_df['classification_accuracy'].values * 100,
        'o-', color='#2ecc71', lw=2, ms=8)
ax.axhline(70, color='red', ls='--', lw=1.5, alpha=0.7, label='GO threshold (70%)')
ax.set_xlabel('Origination Noise (std)', fontsize=11)
ax.set_ylabel('Classification Accuracy (%)', fontsize=11)
ax.set_title('Classification Accuracy vs Noise Level', fontsize=12)
ax.legend(fontsize=9)
ax.grid(True, alpha=0.3)

# --- R-squared vs noise ---
ax = axes[1, 0]
ax.plot(noise_vals, summary_df['r2'].values, 'o-', color='#3498db', lw=2, ms=8)
ax.axhline(0.5, color='orange', ls='--', lw=1.5, alpha=0.7, label='R\u00b2=0.50 (moderate)')
ax.set_xlabel('Origination Noise (std)', fontsize=11)
ax.set_ylabel('R-squared', fontsize=11)
ax.set_title('R-squared vs Noise Level', fontsize=12)
ax.legend(fontsize=9)
ax.grid(True, alpha=0.3)

# --- Within-cluster CV vs noise ---
ax = axes[1, 1]
ax.plot(noise_vals, summary_df['within_cluster_cv'].values,
        'o-', color='#9b59b6', lw=2, ms=8)
ax.set_xlabel('Origination Noise (std)', fontsize=11)
ax.set_ylabel('Mean Within-Cluster CV', fontsize=11)
ax.set_title('Within-Cluster Rent CV vs Noise Level', fontsize=12)
ax.grid(True, alpha=0.3)

plt.suptitle('Simulation 02: Symmetric Distortion Degradation Curves',
             fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

---
## Find the Breaking Point

We define the **breaking point** as the noise level where MAPE first crosses 20%.
If it never crosses within the tested range, we report the pipeline as robust.

We also interpolate linearly between the last passing level and the first failing level
to estimate the precise breaking noise standard deviation.

In [ ]:
# ============================================================
# Find the breaking point (MAPE > 20%)
# ============================================================
BREAK_THRESHOLD = 20.0

mape_values = [r['mape'] for r in results]
noise_values = [r['noise_std'] for r in results]

breaking_idx = None
for i, m in enumerate(mape_values):
    if m >= BREAK_THRESHOLD:
        breaking_idx = i
        break

if breaking_idx is not None:
    breaking_noise = noise_values[breaking_idx]

    # Interpolate more precisely between the level before and the breaking level
    if breaking_idx > 0:
        n_prev = noise_values[breaking_idx - 1]
        m_prev = mape_values[breaking_idx - 1]
        n_curr = noise_values[breaking_idx]
        m_curr = mape_values[breaking_idx]
        interp_noise = n_prev + (
            (n_curr - n_prev) * (BREAK_THRESHOLD - m_prev) / (m_curr - m_prev)
        )
    else:
        interp_noise = breaking_noise

    print(f'BREAKING POINT FOUND')
    print(f'  MAPE crosses {BREAK_THRESHOLD}% at noise_std = {breaking_noise:.2f}')
    print(f'  Interpolated breaking point: noise_std ~ {interp_noise:.3f}')
    print(f'  MAPE at breaking level: {mape_values[breaking_idx]:.1f}%')
    if breaking_idx > 0:
        print(f'  MAPE at previous level: {mape_values[breaking_idx - 1]:.1f}% '
              f'(noise={noise_values[breaking_idx - 1]:.2f})')
else:
    print(f'NO BREAKING POINT FOUND in tested range '
          f'({noise_values[0]:.2f} -- {noise_values[-1]:.2f})')
    print(f'  Maximum MAPE: {max(mape_values):.1f}% at '
          f'noise_std={noise_values[mape_values.index(max(mape_values))]:.2f}')
    print(f'  Pipeline appears robust across all tested noise levels.')
    # Use the highest noise level for defense layer analysis
    breaking_idx = len(results) - 1
    breaking_noise = noise_values[-1]

---
## Defense Layer Analysis at the Breaking Point

We check whether our defense layers -- **Isolation Forest** anomaly detection and
**land value correlation** -- can **detect** when the pipeline is degrading. If defense
metrics degrade in step with model quality, they serve as reliable canaries on real data
where we don't have ground truth.

### Defense Layer 1: Isolation Forest
Run Isolation Forest on features + rent and compare its anomaly labels to the model's
OVERPAYING/UNDERPAYING classification. Agreement is measured by **Cohen's Kappa**.

### Defense Layer 2: Land Value Correlation
Compute **Pearson correlation** between predicted FMV and `underlying_land_value_psf`.
A drop in this correlation signals the model is losing its economic grounding.

In [ ]:
# ============================================================
# Defense layer analysis across ALL noise levels
# ============================================================
defense_metrics = []

for res in results:
    noise_std = res['noise_std']
    df_run = res['df'].copy()
    predicted_fmv = res['predicted_fmv']
    model_class = res['model_class']

    # ----- Defense Layer 1: Isolation Forest -----
    iso_features = df_run[ALL_FEATURES_ENC].fillna(0).copy()
    iso_features['current_monthly_rent'] = df_run['current_monthly_rent'].values

    scaler_iso = StandardScaler()
    X_iso = scaler_iso.fit_transform(iso_features)

    iso_forest = IsolationForest(
        n_estimators=500,
        contamination='auto',
        random_state=42,
        n_jobs=-1,
    )
    iso_labels = iso_forest.fit_predict(X_iso)  # 1=normal, -1=anomaly

    # Map model_class to binary: AT_MARKET -> 1 (normal), else -> -1 (anomaly)
    model_binary = np.where(model_class == 'AT_MARKET', 1, -1)

    kappa = cohen_kappa_score(model_binary, iso_labels)
    agreement = np.mean(model_binary == iso_labels)

    # ----- Defense Layer 2: Land Value Correlation -----
    land_corr, land_p = stats.pearsonr(
        predicted_fmv, df_run['underlying_land_value_psf'].values
    )

    defense_metrics.append({
        'noise_std': noise_std,
        'iso_kappa': kappa,
        'iso_agreement': agreement,
        'land_corr': land_corr,
        'mape': res['mape'],
        'r2': res['r2'],
    })

    print(f'noise={noise_std:.2f}:  Kappa={kappa:.3f}  '
          f'Agreement={agreement:.1%}  Land_corr={land_corr:.3f}  '
          f'MAPE={res["mape"]:.1f}%')

defense_df = pd.DataFrame(defense_metrics)

In [ ]:
# ============================================================
# Plot defense layer degradation alongside model quality
# ============================================================
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# -- Left: MAPE + Kappa (dual axis) --
ax1 = axes[0]
ax1_twin = ax1.twinx()

l1, = ax1.plot(defense_df['noise_std'], defense_df['mape'],
               'o-', color='#e74c3c', lw=2, ms=7, label='MAPE (%)')
l2, = ax1_twin.plot(defense_df['noise_std'], defense_df['iso_kappa'],
                    's-', color='#3498db', lw=2, ms=7, label="Cohen's Kappa")

ax1.set_xlabel('Origination Noise (std)', fontsize=11)
ax1.set_ylabel('MAPE (%)', fontsize=11, color='#e74c3c')
ax1_twin.set_ylabel("Cohen's Kappa", fontsize=11, color='#3498db')
ax1.set_title('MAPE vs Isolation Forest Kappa', fontsize=12)
ax1.legend(handles=[l1, l2], loc='upper left', fontsize=9)
ax1.grid(True, alpha=0.3)

# -- Center: MAPE + Land Value Correlation (dual axis) --
ax2 = axes[1]
ax2_twin = ax2.twinx()

l3, = ax2.plot(defense_df['noise_std'], defense_df['mape'],
               'o-', color='#e74c3c', lw=2, ms=7, label='MAPE (%)')
l4, = ax2_twin.plot(defense_df['noise_std'], defense_df['land_corr'],
                    'D-', color='#2ecc71', lw=2, ms=7, label='Land Value Corr')

ax2.set_xlabel('Origination Noise (std)', fontsize=11)
ax2.set_ylabel('MAPE (%)', fontsize=11, color='#e74c3c')
ax2_twin.set_ylabel('Pearson r (FMV vs Land Value)', fontsize=11, color='#2ecc71')
ax2.set_title('MAPE vs Land Value Correlation', fontsize=12)
ax2.legend(handles=[l3, l4], loc='upper left', fontsize=9)
ax2.grid(True, alpha=0.3)

# -- Right: All defense metrics normalized for co-degradation --
ax3 = axes[2]
norm_data = defense_df[['mape', 'iso_kappa', 'land_corr']].copy()
# Normalize: higher = worse degradation
norm_data['mape'] = norm_data['mape'] / norm_data['mape'].max()
norm_data['iso_kappa'] = 1 - norm_data['iso_kappa'] / max(
    norm_data['iso_kappa'].max(), 0.01
)
norm_data['land_corr'] = 1 - norm_data['land_corr'] / max(
    norm_data['land_corr'].max(), 0.01
)

ax3.plot(defense_df['noise_std'], norm_data['mape'],
         'o-', color='#e74c3c', lw=2, ms=7, label='MAPE (norm)')
ax3.plot(defense_df['noise_std'], norm_data['iso_kappa'],
         's-', color='#3498db', lw=2, ms=7, label='1 - Kappa (norm)')
ax3.plot(defense_df['noise_std'], norm_data['land_corr'],
         'D-', color='#2ecc71', lw=2, ms=7, label='1 - Land Corr (norm)')
ax3.set_xlabel('Origination Noise (std)', fontsize=11)
ax3.set_ylabel('Normalized Degradation (0=good, 1=bad)', fontsize=11)
ax3.set_title('Defense Layer Co-degradation', fontsize=12)
ax3.legend(fontsize=9)
ax3.grid(True, alpha=0.3)

plt.suptitle('Simulation 02: Defense Layer Analysis', fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

---
## Full Summary Table (Model + Defense Metrics)

This combined table shows pipeline performance metrics alongside defense layer metrics
at each noise level, providing a complete picture of how the system degrades and whether
the defense layers track that degradation.

In [ ]:
# ============================================================
# Combined summary table
# ============================================================
full_summary = summary_df.merge(
    defense_df[['noise_std', 'iso_kappa', 'iso_agreement', 'land_corr']],
    on='noise_std',
)

# Format for display
disp = full_summary.copy()
disp['mape'] = disp['mape'].map('{:.1f}%'.format)
disp['mae'] = disp['mae'].map('${:,.0f}'.format)
disp['rmse'] = disp['rmse'].map('${:,.0f}'.format)
disp['r2'] = disp['r2'].map('{:.3f}'.format)
disp['classification_accuracy'] = disp['classification_accuracy'].map('{:.1%}'.format)
disp['core_pct'] = disp['core_pct'].map('{:.1%}'.format)
disp['within_cluster_cv'] = disp['within_cluster_cv'].map('{:.3f}'.format)
disp['iso_kappa'] = disp['iso_kappa'].map('{:.3f}'.format)
disp['iso_agreement'] = disp['iso_agreement'].map('{:.1%}'.format)
disp['land_corr'] = disp['land_corr'].map('{:.3f}'.format)

print('=' * 120)
print('SIMULATION 02: FULL SUMMARY -- SYMMETRIC DISTORTION STRESS TEST')
print('=' * 120)
print(disp.to_string(index=False))

---
## Conclusions

This section synthesizes findings across all noise levels:
1. **Breaking point**: At what noise level does MAPE exceed 20%?
2. **Degradation pattern**: How does each metric degrade?
3. **Defense layer effectiveness**: Do defense metrics co-degrade with model quality?
4. **Bottom line**: Is the pipeline robust enough for real-world Verizon data?

In [ ]:
# ============================================================
# Final conclusion
# ============================================================
mape_vals = [r['mape'] for r in results]
noise_vals_list = [r['noise_std'] for r in results]
kappa_vals = defense_df['iso_kappa'].values
land_vals = defense_df['land_corr'].values

print('=' * 70)
print('SIMULATION 02: CONCLUSIONS')
print('=' * 70)

# 1. Breaking point
broken = [i for i, m in enumerate(mape_vals) if m >= 20.0]
if broken:
    bp_idx = broken[0]
    print(f'\n1. BREAKING POINT:')
    print(f'   Pipeline breaks (MAPE >= 20%) at noise_std = {noise_vals_list[bp_idx]:.2f}')
    print(f'   MAPE at breaking point: {mape_vals[bp_idx]:.1f}%')
    if bp_idx > 0:
        print(f'   Last acceptable level: noise_std = {noise_vals_list[bp_idx - 1]:.2f} '
              f'(MAPE = {mape_vals[bp_idx - 1]:.1f}%)')
else:
    print(f'\n1. BREAKING POINT:')
    print(f'   Pipeline does NOT break across tested range.')
    print(f'   Maximum MAPE: {max(mape_vals):.1f}% at '
          f'noise_std = {noise_vals_list[mape_vals.index(max(mape_vals))]:.2f}')

# 2. Degradation pattern
print(f'\n2. DEGRADATION PATTERN:')
for i, (n, m) in enumerate(zip(noise_vals_list, mape_vals)):
    status = 'PASS' if m < 15 else ('WARN' if m < 20 else 'FAIL')
    print(f'   noise={n:.2f}: MAPE={m:.1f}%  [{status}]')

# 3. Defense layer effectiveness
print(f'\n3. DEFENSE LAYER EFFECTIVENESS:')
if len(mape_vals) >= 3:
    mape_kappa_corr = np.corrcoef(mape_vals, kappa_vals)[0, 1]
    mape_land_corr = np.corrcoef(mape_vals, land_vals)[0, 1]
    print(f'   Correlation(MAPE, Kappa):      {mape_kappa_corr:.3f}  '
          f'({"STRONG" if abs(mape_kappa_corr) > 0.7 else "WEAK"} co-movement)')
    print(f'   Correlation(MAPE, Land_corr):  {mape_land_corr:.3f}  '
          f'({"STRONG" if abs(mape_land_corr) > 0.7 else "WEAK"} co-movement)')

    if abs(mape_kappa_corr) > 0.7 or abs(mape_land_corr) > 0.7:
        print(f'   -> Defense layers DETECT degradation '
              f'(strong co-movement with MAPE)')
    else:
        print(f'   -> Defense layers may NOT reliably detect degradation')
else:
    print(f'   Insufficient data points for correlation analysis.')

# 4. Bottom line
print(f'\n4. BOTTOM LINE:')
go_levels = [n for n, m in zip(noise_vals_list, mape_vals) if m < 15]
if go_levels:
    print(f'   Pipeline meets GO criteria (MAPE<15%) at noise_std <= {max(go_levels):.2f}')
else:
    print(f'   Pipeline does not meet GO criteria at any tested noise level.')
print(f'   Real-world Verizon data has estimated origination noise ~ 0.10-0.15')
safety_ok = max(go_levels, default=0) >= 0.15 if go_levels else False
print(f'   Safety margin: {"ADEQUATE" if safety_ok else "NARROW or INSUFFICIENT"}')